## Vision Transformer (ViT)

In this assignment we're going to work with Vision Transformer. We will start to build our own vit model and train it on an image classification task.
The purpose of this homework is for you to get familar with ViT and get prepared for the final project.

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# VIT Implementation

The vision transformer can be seperated into three parts, we will implement each part and combine them in the end.

For the implementation, feel free to experiment different kinds of setup, as long as you use attention as the main computation unit and the ViT can be train to perform the image classification task present later.
You can read about the ViT implement from other libary: https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py and https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py

## PatchEmbedding
PatchEmbedding is responsible for dividing the input image into non-overlapping patches and projecting them into a specified embedding dimension. It uses a 2D convolution layer with a kernel size and stride equal to the patch size. The output is a sequence of linear embeddings for each patch.

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
      super().__init__()
      self.embed_dim = embed_dim
      self.patch_layer = torch.nn.Conv2d(in_channels=in_channels, out_channels=embed_dim,
                                   kernel_size=patch_size, stride=patch_size)
      num_patches = (image_size // patch_size) ** 2
      self.position = nn.Parameter(torch.randn(num_patches+ 1, embed_dim))

    def forward(self, x):
      x = self.patch_layer(x)
      x = x.flatten(2).transpose(1, 2)

      batch_size, _, _ = x.size()
      cls_token = nn.Parameter(torch.randn(1, 1, self.embed_dim))
      cls_token = cls_token.expand(batch_size, -1, -1).to(device)

      x = torch.cat([cls_token, x], dim = 1)
      x += self.position

      return x

## MultiHeadSelfAttention

This class implements the multi-head self-attention mechanism, which is a key component of the transformer architecture. It consists of multiple attention heads that independently compute scaled dot-product attention on the input embeddings. This allows the model to capture different aspects of the input at different positions. The attention outputs are concatenated and linearly transformed back to the original embedding size.

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
      super().__init__()
      self.embed_dim = embed_dim
      self.num_heads = num_heads

      self.head_dim = embed_dim // num_heads

      self.Wq = torch.nn.Linear(embed_dim, embed_dim)
      self.Wk = torch.nn.Linear(embed_dim, embed_dim)
      self.Wv = torch.nn.Linear(embed_dim, embed_dim)
      self.Wo = torch.nn.Linear(embed_dim, embed_dim)


    def forward(self, x):
      batch_size, num_patches, embed_dim = x.shape

      queries = self.Wq(x)
      queries = queries.view(batch_size, num_patches, self.num_heads,
                             self.head_dim).permute(0, 2, 3, 1)

      keys = self.Wk(x)
      keys = keys.view(batch_size, num_patches, self.num_heads,
                       self.head_dim).permute(0, 2, 3, 1)

      values = self.Wv(x)
      values = values.view(batch_size, num_patches, self.num_heads,
                           self.head_dim).permute(0, 2, 3, 1)

      attn_scores = (queries @ keys.transpose(-2, -1)) / (self.head_dim ** 0.5)
      attn_weights = F.softmax(attn_scores, dim=-1)

      output = attn_weights @ values
      output = output.permute(0, 3, 1, 2)
      output = output.contiguous().view(batch_size, num_patches, embed_dim)
      output = self.Wo(output)

      return output

## TransformerBlock
This class represents a single transformer layer. It includes a multi-head self-attention sublayer followed by a position-wise feed-forward network (MLP). Each sublayer is surrounded by residual connections.
You may also want to use layer normalization or other type of normalization.

In [ ]:
class MLP(nn.Module):
    def __init__(self, embed_dim, mlp_dim, dropout):
        super().__init__()
        self.layer1 = nn.Linear(embed_dim, mlp_dim)
        self.gelu = nn.GELU()
        self.layer2 = nn.Linear(mlp_dim, embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer1(x)
        x = self.gelu(x)
        x = self.layer2(x)
        x = self.dropout(x)
        return x

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
      super().__init__()
      self.attention = MultiHeadSelfAttention(embed_dim=embed_dim,
                                         num_heads=num_heads)
      self.layer_norm1 = torch.nn.LayerNorm(embed_dim)
      self.mlp = MLP(embed_dim, mlp_dim, dropout=dropout)
      self.layer_norm2 = torch.nn.LayerNorm(embed_dim)

    def forward(self, x):
      attention = self.attention(self.layer_norm1(x))
      x = x + attention

      mlp_out = self.mlp(self.layer_norm2(x))
      x = x + mlp_out

      return x

## VisionTransformer:
This is the main class that assembles the entire Vision Transformer architecture. It starts with the PatchEmbedding layer to create patch embeddings from the input image. A special class token is added to the sequence, and positional embeddings are added to both the patch and class tokens. The sequence of patch embeddings is then passed through multiple TransformerBlock layers. The final output is the logits for all classes

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout=0.1):
      super().__init__()
      self.patch_embedding = PatchEmbedding(image_size, patch_size, in_channels, embed_dim)
      self.transformers = nn.ModuleList([
          TransformerBlock(embed_dim, num_heads, mlp_dim, dropout)
          for _ in range(num_layers)
      ])
      self.classifiers = torch.nn.Linear(embed_dim, num_classes)

    def forward(self, x):
      embedded = self.patch_embedding(x)
      for transformer in self.transformers:
        embedded = transformer(embedded)

      encoded = embedded.mean(dim=1)
      classified = self.classifiers(encoded)
      return classified

## Let's train the ViT!

We will train the vit to do the image classification with cifar100. Free free to change the optimizer and or add other tricks to improve the training

In [35]:
# Example usage:
image_size = 32 # TODO
patch_size = 4 # TODO
in_channels = 3 # TODO
embed_dim = 48 # TODO
num_heads = 4 # TODO
mlp_dim = 192 # TODO
num_layers = 4 # TODO
num_classes = 100 # TODO
dropout = 0.1 # TODO

batch_size = 128 # TODO

In [36]:
model = VisionTransformer(image_size, patch_size, in_channels, embed_dim, num_heads, mlp_dim, num_layers, num_classes, dropout).to(device)
input_tensor = torch.randn(1, in_channels, image_size, image_size).to(device)
output = model(input_tensor)
print(output.shape)

torch.Size([1, 100])


In [37]:
# Load the CIFAR-100 dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [38]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

In [39]:
# Train the model
from tqdm import tqdm

num_epochs = 150 # TODO
best_val_acc = 0
for epoch in range(num_epochs):
    model.train()
    for i, data in enumerate(tqdm(trainloader, desc=f'Epoch {epoch+1}/{num_epochs}')):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # TODO Feel free to modify the training loop youself.

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Epoch: {epoch + 1}, Validation Accuracy: {val_acc:.2f}%")

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

Epoch 1/150: 100%|██████████| 391/391 [00:12<00:00, 31.99it/s]


Epoch: 1, Validation Accuracy: 12.79%


Epoch 2/150: 100%|██████████| 391/391 [00:12<00:00, 31.71it/s]


Epoch: 2, Validation Accuracy: 15.79%


Epoch 3/150: 100%|██████████| 391/391 [00:12<00:00, 32.52it/s]


...

Epoch: 148, Validation Accuracy: 49.09%


Epoch 149/150: 100%|██████████| 391/391 [00:13<00:00, 30.03it/s]


Epoch: 149, Validation Accuracy: 51.31%


Epoch 150/150: 100%|██████████| 391/391 [00:13<00:00, 29.28it/s]


Epoch: 150, Validation Accuracy: 47.74%


Please submit your best_model.pth with this notebook. And report the best test results you get: Maximum accuracy is 51.31% if we utilize early-stopping.